## Generate data for the screw extrusion process model

In [1]:
import jax
import numpy as np
from jax import random
from tqdm import trange, tqdm
import matplotlib.pyplot as plt

import os
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'false'
from SCREW import screwModel
IMPORT_PATH = os.getcwd()
print(IMPORT_PATH)
%matplotlib inline
import gc
gc.collect()

from scipy.stats.qmc import LatinHypercube


/home/lab422/422File/Class_2023/zt/2025/Adaptivedesign10/data
/home/lab422/422File/Class_2023/zt/2025/Adaptivedesign10/data


In [ ]:
#################模型求解#################
Z=200
tmax=200
dt=1
N_train = 3000#1000 # number of input samples
N_test = 500#500
# Select N keys to create N Functions
key = random.PRNGKey(0)
keys = random.split(key, N_train)




In [ ]:
def generate_lhs_samples(n_params=3, n_samples=N_train, param_ranges=None, seed=42):
    """
    :param n_params: 
    :param n_samples: 
    :param param_ranges: 
    :param seed: 
    :return: 
    """

    if len(param_ranges) != n_params:
        raise ValueError("参数范围列表的长度必须与参数数量相等")
    
    samples = np.zeros((n_samples, n_params))  

    rng = np.random.default_rng(seed)
    seeds = rng.integers(0, 1e6, size=n_params)  
    
    for i, (lower, upper) in enumerate(param_ranges):
        sampler = LatinHypercube(d=1, seed=seeds[i])  
        raw_samples = sampler.random(n_samples).flatten() 
        scaled_samples = lower + (upper - lower) * raw_samples  
        samples[:, i] = scaled_samples  
    
    return samples



param_ranges = [(1e-3, 2e-1), (1e-2, 3), (1, 30)]
seed = 111
samples = generate_lhs_samples(param_ranges=param_ranges, n_samples=N_train, seed=seed)


采样结果形状: (3000, 3)


In [ ]:


samples_split = [samples[:, i] for i in range(samples.shape[1])]#<class 'list'>


expanded_samples = [np.tile(sample, (Z, 1)).T for sample in samples_split]#<class 'list'>
mu1s, mu2s, alphas= expanded_samples

# 打印结果，检查每个数组的维度
for i, arr in enumerate(expanded_samples):
    print(f"Array {i+1} shape: {arr.shape}")





Array 1 shape: (3000, 200)
Array 2 shape: (3000, 200)
Array 3 shape: (3000, 200)


## Training Data




We need to generate 5000 functions, so lets create a function to help us with the training and testing data


In [ ]:
# Geneate training data corresponding to one input sample
def generate_one_training_data(key,Z,tmax,dt,param):
    mu1, mu2, alpha = param
    # Numerical solution
    model = screwModel(key,Z,tmax,dt,mu1, mu2, alpha)
    xx,tt,fp,T,lt0,sensor_values1,sensor_values2,sensor_values3 = model.solve()
    del model
    gc.collect()  
    return fp,T,lt0,sensor_values1,sensor_values2,sensor_values3


In [6]:
sensor_values1s = []
sensor_values2s = []
sensor_values3s = []
fps = []
Ts  = []
lts = []
# Sequentially generate data
for i in tqdm(range(N_train), desc="Generating Data", ncols=80,unit="step", unit_scale=True):
    
    fp,T,lt0,sensor_values1, sensor_values2,sensor_values3 = generate_one_training_data(keys[i],Z,tmax,dt,samples[i, :])
    fps.append(fp)
    lts.append(lt0)
    Ts.append(T)
    sensor_values1s.append(sensor_values1)
    sensor_values2s.append(sensor_values2)
    sensor_values3s.append(sensor_values3)
    

Generating Data: 100%|████████████████| 3.00k/3.00k [15:08:45<00:00, 18.2s/step]


In [ ]:
# Convert lists to numpy arrays for consistency
sensor_values1s = np.array(sensor_values1s)
sensor_values2s = np.array(sensor_values2s)
sensor_values3s = np.array(sensor_values3s)

Ts = np.array(Ts)#Ts.shape= (N, 200, 200)
fps = np.array(fps)
lts = np.array(lts)

Ts = np.reshape(Ts, (-1, Z*tmax))#
fps = np.reshape(fps, (-1, Z*tmax))
lts_expanded = np.expand_dims(lts, axis=2)#
lts_repeated = np.tile(lts_expanded, (1, 1, 200))
ltss = np.reshape(lts_repeated, (-1, Z*tmax))#(N,Z*tmax)


In [ ]:
file_name="Train_3000.npz"
save_path = os.path.join(IMPORT_PATH, file_name)
np.savez_compressed(save_path, sensor_values1=sensor_values1s, sensor_values2=sensor_values2s, sensor_values3=sensor_values3s, Ts=Ts,\
                    fps=fps,ltss=ltss,mu1s=mu1s, mu2s=mu2s, alphas=alphas)


In [ ]:
x=np.linspace(0, 2, Z+1)
space=x[:-1]
time = np.arange(0, tmax , dt)
xt = [(x0, y0) for x0 in space for y0 in time]
np.savez_compressed("xt.npz", xt=xt)




## TESTING DATA

In [ ]:
key = random.PRNGKey(1)
keys = random.split(key, N_test)
sensor_values1s0 = []
sensor_values2s0 = []
sensor_values3s0 = []
fps0= []
Ts0 = []
lts0=[]

param_ranges = [(1e-3, 2e-1), (1e-2, 3), (1, 30)]
seed = 122
samples_test = generate_lhs_samples(param_ranges=param_ranges, n_samples=N_test, seed=seed)


samples_split2 = [samples_test[:, i] for i in range(samples_test.shape[1])]#<class 'list'>

expanded_samples2 = [np.tile(samples_test, (Z, 1)).T for samples_test in samples_split2]#<class 'list'>
mu1s2, mu2s2, alphas2= expanded_samples2

for i in tqdm(range(N_test), desc="Generating Test Data", ncols=80,unit="step", unit_scale=True):

    fp,T,lt0,sensor_values1, sensor_values2,sensor_values3 = generate_one_training_data(keys[i],Z,tmax,dt,samples_test[i,:])
    fps0.append(fp)
    lts0.append(lt0)
    sensor_values1s0.append(sensor_values1)
    sensor_values2s0.append(sensor_values2)
    sensor_values3s0.append(sensor_values3)
    Ts0.append(T)

# Convert lists to numpy arrays for consistency
sensor_values1s0 = np.array(sensor_values1s0)
sensor_values2s0 = np.array(sensor_values2s0)
sensor_values3s0 = np.array(sensor_values3s0)
fps0=np.array(fps0)
Ts0 = np.array(Ts0)
lts0=np.array(lts0)
Ts0 = np.reshape(Ts0, (-1, Z*tmax))

Ts0 = np.reshape(Ts0, (-1, Z*tmax))
fps0 = np.reshape(fps0, (-1, Z*tmax))
lts0_expanded = np.expand_dims(lts0, axis=2)
lts0_repeated = np.tile(lts0_expanded, (1, 1, 200))
ltss0 = np.reshape(lts0_repeated, (-1, Z*tmax))#(N,Z*tmax)



Generating Test Data: 100%|████████████████| 500/500 [2:36:31<00:00, 18.8s/step]


In [ ]:

file_name="Test_500.npz"
save_path = os.path.join(IMPORT_PATH, file_name)
np.savez_compressed(save_path, sensor_values1=sensor_values1s0, sensor_values2=sensor_values2s0,sensor_values3=sensor_values3s0,\
                    Ts=Ts0,fps=fps0,ltss=ltss0,mu1s=mu1s2, mu2s=mu2s2, alphas=alphas2)

print("sensor_values1s0.shape= ",sensor_values1s0.shape)
print("ltss0.shape= ",ltss0.shape)

sensor_values1s0.shape=  (500, 200)
ltss0.shape=  (500, 40000)
